## Install Requirements

In [1]:
!pip install numpy
!pip install opencv-python
!pip install tqdm
!pip install Kaggle
!pip install mediapipe
# !pip install mediapipe==0.8.9

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 33.0 MB 133.9 MB/s 
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires flatbuffers<2,>=1.12, but you have flatbuffers 22.11.23 which is incompatib

##mount the google drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##import requirements

In [3]:
import numpy as np
import os
import pandas as pd
import pickle
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
import cv2
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import PIL
import io
import html
import time
import mediapipe as mp
from google.colab.patches import cv2_imshow

##download dataset from kaggle

In [ ]:
# kaggle configuration
os.getcwd()
# Enable the Kaggle environment, use the path to the directory your Kaggle API JSON is stored in
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/kaggle'

In [ ]:
# download the kaggle dataset
os.chdir('/content/drive/MyDrive/kaggle')
!kaggle datasets download -d elysian01/yoga-pose-classification
os.listdir()

FileNotFoundError: ignored

In [ ]:
from zipfile import ZipFile 
  
# specifying the name of the zip file
file = "/content/drive/MyDrive/kaggle/yoga-pose-classification.zip"
  
# open the zip file in read mode
with ZipFile(file, 'r') as zip: 
    # extract all files to another directory
    zip.extractall('/content/drive/MyDrive/yoga_pose_dataset')


##defining folders path

In [ ]:
images_in_folder = '/content/drive/MyDrive/yoga_pose_dataset/dataset/yoga_set1/test'
images_out_folder = '/content/drive/MyDrive/yoga_pose_dataset/test_dataset'
csv_out_path = '/content/drive/MyDrive/yoga_pose_dataset/test_dataset/yoga_test.csv'

##dataset creation using mindpipe

In [ ]:
import csv
import cv2
import numpy as np
import os
import sys
import tqdm

from mediapipe.python.solutions import drawing_utils as mp_drawing
from mediapipe.python.solutions import pose as mp_pose


with open(csv_out_path, 'w') as csv_out_file:
  csv_out_writer = csv.writer(csv_out_file, delimiter=',', quoting=csv.QUOTE_MINIMAL)

  # Folder names are used as pose class names.
  pose_class_names = sorted([n for n in os.listdir(images_in_folder) if not n.startswith('.')])

  for pose_class_name in pose_class_names:
    print('Bootstrapping ', pose_class_name, file=sys.stderr)

    if not os.path.exists(os.path.join(images_out_folder, pose_class_name)):
      os.makedirs(os.path.join(images_out_folder, pose_class_name))

    image_names = sorted([
        n for n in os.listdir(os.path.join(images_in_folder, pose_class_name))
        if not n.startswith('.')])
    for image_name in tqdm.tqdm(image_names, position=0):
      # Load image.
      input_frame = cv2.imread(os.path.join(images_in_folder, pose_class_name, image_name))
      input_frame = cv2.cvtColor(input_frame, cv2.COLOR_BGR2RGB)

      # Initialize fresh pose tracker and run it.
      with mp_pose.Pose(upper_body_only=False) as pose_tracker:
        result = pose_tracker.process(image=input_frame)
        pose_landmarks = result.pose_landmarks
      
      # Save image with pose prediction (if pose was detected).
      output_frame = input_frame.copy()
      if pose_landmarks is not None:
        mp_drawing.draw_landmarks(
            image=output_frame,
            landmark_list=pose_landmarks,
            connections=mp_pose.POSE_CONNECTIONS)
      output_frame = cv2.cvtColor(output_frame, cv2.COLOR_RGB2BGR)
      cv2.imwrite(os.path.join(images_out_folder, image_name), output_frame)
      
      # Save landmarks.
      if pose_landmarks is not None:
        # Check the number of landmarks and take pose landmarks.
        assert len(pose_landmarks.landmark) == 33, 'Unexpected number of predicted pose landmarks: {}'.format(len(pose_landmarks.landmark))
        pose_landmarks = [[lmk.x, lmk.y, lmk.z] for lmk in pose_landmarks.landmark]

        # Map pose landmarks from [0, 1] range to absolute coordinates to get
        # correct aspect ratio.
        frame_height, frame_width = output_frame.shape[:2]
        pose_landmarks *= np.array([frame_width, frame_height, frame_width])

        # Write pose sample to CSV.
        pose_landmarks = np.around(pose_landmarks, 5).flatten().astype(np.str).tolist()
        csv_out_writer.writerow([image_name, pose_class_name] + pose_landmarks)


In [ ]:
df_test = pd.read_csv("/content/drive/MyDrive/yoga_pose_dataset/yoga_test.csv")
df_test.head()

FileNotFoundError: ignored

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/yoga_pose_dataset/yoga_csv.csv")
df.head()

##calculate angle for 3 points

In [ ]:
# to calculate angle in three points
def cal_ang(item, pt1, pt2, pt3):
  # print(item)
  a = np.array([item[(pt1*3)+2], item[(pt1*3)+1+2], item[(pt1*3)+2+2]])
  b = np.array([item[(pt2*3)+2], item[(pt2*3)+1+2], item[(pt2*3)+2+2]])
  c = np.array([item[(pt3*3)+2], item[(pt3*3)+1+2], item[(pt3*3)+2+2]])

  ba = a - b
  bc = c - b

  cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
  angle = (np.arccos(cosine_angle))*(180/np.pi)

  return angle

##calculate angles to define body features

In [ ]:
# creating dataframe for feature angles
df_angle = pd.DataFrame(columns=['left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow', 'left_leg_waist', 'right_leg_waist', 'left_knee', 'right_knee', 'target'])
df_angle_test = pd.DataFrame(columns=['left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow', 'left_leg_waist', 'right_leg_waist', 'left_knee', 'right_knee', 'target'])

In [ ]:
# filling the dataframes
for index, item in df.iterrows():
  # a = np.array([item[(13*3)], item[(13*3)+1], item[(13*3)+2]])
  # a = np.array([item[(13*3)], item[(13*3)+1], item[(13*3)+2]])
  dict = {
      'left_shoulder': cal_ang(item, 13, 11, 12),
      'right_shoulder': cal_ang(item, 24, 12, 14),
      'left_elbow': cal_ang(item, 15, 13, 11),
      'right_elbow': cal_ang(item, 12, 14, 16),
      'left_leg_waist': cal_ang(item, 11, 23, 25),
      'right_leg_waist': cal_ang(item, 12, 24, 26),
      'left_knee': cal_ang(item, 23, 25, 27),
      'right_knee': cal_ang(item, 24, 26, 28),
      'target': item[1]
  }
  # print("index = ", index)
  # for i in range(0, len(item)):
  #   print(item[i])
  # if(index==2):
  #   break
  # print("done")  
  df_angle = df_angle.append(dict, ignore_index=True)

In [ ]:
for index, item in df_test.iterrows():
  # a = np.array([item[(13*3)], item[(13*3)+1], item[(13*3)+2]])
  # a = np.array([item[(13*3)], item[(13*3)+1], item[(13*3)+2]])
  dict = {
      'left_shoulder': cal_ang(item, 13, 11, 12),
      'right_shoulder': cal_ang(item, 24, 12, 14),
      'left_elbow': cal_ang(item, 15, 13, 11),
      'right_elbow': cal_ang(item, 12, 14, 16),
      'left_leg_waist': cal_ang(item, 11, 23, 25),
      'right_leg_waist': cal_ang(item, 12, 24, 26),
      'left_knee': cal_ang(item, 23, 25, 27),
      'right_knee': cal_ang(item, 24, 26, 28),
      'target': item[1]
  }
  # print("index = ", index)
  # for i in range(0, len(item)):
  #   print(item[i])
  # if(index==2):
  #   break
  # print("done")  
  df_angle_test = df_angle_test.append(dict, ignore_index=True)

In [23]:
df_angle = pd.read_csv('/content/drive/MyDrive/yoga_angle.csv')
df_angle_test = pd.read_csv('/content/drive/MyDrive/yoga_angle_test.csv')

In [24]:
df_angle[df_angle['target'] == 'downdog']

,Unnamed: 0,left_shoulder,right_shoulder,left_elbow,right_elbow,left_leg_waist,right_leg_waist,left_knee,right_knee,target
0,0,143.176041,147.824399,114.986452,93.792486,71.981782,64.813235,173.639815,156.346628,downdog
1,1,139.288780,145.156636,131.170793,99.637033,75.943327,70.144649,168.635117,155.601473,downdog
2,2,116.616909,173.894784,91.117305,164.415748,83.022949,69.420660,167.002745,159.490200,downdog
3,3,145.895066,123.270036,132.198656,53.619093,100.751834,82.769971,172.951286,167.115766,downdog
4,4,144.398010,140.334221,57.816453,148.502738,83.287876,93.894641,164.633229,162.323169,downdog
...,...,...,...,...,...,...,...,...,...,...
343,343,135.678797,135.181224,110.274623,88.645270,103.180871,101.934329,169.324783,155.328283,downdog
344,344,139.518199,130.788954,106.686327,92.527192,89.816328,87.774871,175.316490,158.004182,downdog
345,345,142.607392,53.311210,76.823388,120.822136,141.337974,161.596687,142.603460,110.190861,downdog
346,346,120.475854,126.337124,169.342488,115.492176,104.342722,109.894360,158.500639,158.845581,downdog


In [25]:
df_angle_test.drop('Unnamed: 0', inplace=True, axis=1)
df_angle.drop('Unnamed: 0', inplace=True, axis=1)

##training

In [26]:
# describe features
import matplotlib.pyplot as mtp  
  
#Extracting Independent and dependent Variable  
x_train= df_angle.iloc[:, 0:8].values  
y_train= df_angle.iloc[:, 8].values  
x_test = df_angle_test.iloc[:, 0:8].values 
y_test = df_angle_test.iloc[:, 8].values    
  
#feature Scaling  
# from sklearn.preprocessing import StandardScaler    
# st_x= StandardScaler()    
# x_train= st_x.fit_transform(x_train)    
# x_test= st_x.transform(x_test)  

In [27]:
df_angle_test.head(10)

,left_shoulder,right_shoulder,left_elbow,right_elbow,left_leg_waist,right_leg_waist,left_knee,right_knee,target
0,153.889524,138.894707,137.569890,76.171640,76.822979,68.322260,173.676172,147.484575,downdog
1,148.984172,144.510751,118.134282,77.656030,76.177528,71.451253,176.418239,148.038165,downdog
2,137.983160,140.481900,64.367022,129.185550,71.580972,78.080602,162.244509,176.341972,downdog
3,139.447815,133.380929,64.317826,112.953782,73.168204,75.364128,154.374658,173.976926,downdog
4,81.959654,127.233231,141.941720,172.948443,46.664656,77.816850,145.384025,151.499912,downdog
5,141.060937,136.226006,60.707847,107.140378,82.602151,85.589903,152.232854,166.499219,downdog
6,126.385929,151.271193,77.922776,111.495118,89.125094,88.973821,151.187347,161.166915,downdog
7,151.515936,118.221190,148.060578,67.930635,100.861391,92.366500,166.038336,158.763603,downdog
8,149.346339,143.620450,95.503059,68.357336,87.104440,80.922777,173.437245,151.521916,downdog
9,151.283501,132.944869,110.544668,63.279992,83.153946,79.625148,165.349480,146.828186,downdog


In [46]:
# define the model
from sklearn.neighbors import KNeighborsClassifier  
classifier= KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2 )  
classifier.fit(x_train, y_train)  

KNeighborsClassifier()

In [47]:
# predicting
y_pred= classifier.predict(x_test)  

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf = clf.fit(x_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(x_test)

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb = gnb.fit(x_train, y_train)
  
# making predictions on the testing set
y_pred = gnb.predict(x_test)

In [32]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators = 100) 
clf.fit(x_train, y_train)
 
# performing predictions on the test dataset
y_pred = clf.predict(x_test)

In [36]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
clf.fit(x_train, y_train)
 
# performing predictions on the test dataset
y_pred = clf.predict(x_test)

In [40]:
from xgboost import XGBClassifier
clf = XGBClassifier()
clf.fit(x_train, y_train)

# performing predictions on the test dataset
y_pred = clf.predict(x_test)

In [42]:
from sklearn import svm
clf = svm.SVC()
clf.fit(x_train, y_train)
 
# performing predictions on the test dataset
y_pred = clf.predict(x_test)

In [44]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(random_state=1, max_iter=500)
clf.fit(x_train, y_train)

# performing predictions on the test dataset
y_pred = clf.predict(x_test)

In [37]:
# confusion matrix
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test, y_pred) 

In [38]:
cm

array([[91,  0,  0,  0,  2,  0],
       [ 0, 66,  1,  1,  2, 10],
       [ 0,  0, 26,  3,  1,  0],
       [ 1,  2, 13, 49,  1,  2],
       [ 0,  0,  1,  0,  2,  1],
       [ 0,  7,  0,  0,  4, 96]])

In [12]:
# testing for individual test
classifier.predict([x_test[100]])

NameError: ignored

In [48]:
# calculating accuracy
from sklearn.metrics import accuracy_score  
acc = accuracy_score(y_test, y_pred)
acc

0.9450261780104712

In [ ]:
# df to csv
df_angle.to_csv("yoga_angle.csv", index=False)
df_angle_test.to_csv("yoga_angle_test.csv", index = False)

In [ ]:

def plot_correlation(data):
    '''
    plot correlation's matrix to explore dependency between features 
    '''
    # init figure size
    rcParams['figure.figsize'] = 15, 20
    fig = plt.figure()
    sns.heatmap(data.corr(), annot=True, fmt=".2f")
    plt.show()
    fig.savefig('corr.png')

# load your data 
# data  = pd.read_csv('diabetes.csv')

# plot correlation & densities
plot_correlation(df_angle)

#dump the model

In [ ]:
# dump the model
pickle.dump(classifier,open('save_model3','wb'))


#load the model

In [ ]:
# load the model
classifier = pickle.load(open('/content/drive/MyDrive/yoga_pose_dataset/save_model3', 'rb'))

#testing


In [ ]:
# angle calculate for test features
def cal_ang_test(item, pt1, pt2, pt3):
  # print(item)
  # a = np.array([item[pt1][0], item[pt1][1], item[pt1][2]])
  # b = np.array([item[pt2][0], item[pt2][1], item[pt2][2]])
  # c = np.array([item[pt3][0], item[pt3][1], item[pt3][2]])
  a = item[pt1]
  b = item[pt2]
  c = item[pt3]
  ba = a - b
  bc = c - b

  cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
  angle = (np.arccos(cosine_angle))*(180/np.pi)

  return angle

In [ ]:
import cv2
from mediapipe.python.solutions import drawing_utils as mp_drawing
from mediapipe.python.solutions import pose as mp_pose
input_frame = cv2.imread('/content/baai4.jpg')
input_frame = cv2.cvtColor(input_frame, cv2.COLOR_BGR2RGB)
with mp_pose.Pose(upper_body_only=False) as pose_tracker:
  result = pose_tracker.process(image=input_frame)
  # mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)
  pose_landmarks = result.pose_landmarks
output_frame = input_frame.copy()
if pose_landmarks is not None:
  mp_drawing.draw_landmarks(
      image=output_frame,
      landmark_list=pose_landmarks,
      connections=mp_pose.POSE_CONNECTIONS)
  assert len(pose_landmarks.landmark) == 33, 'Unexpected number of predicted pose landmarks: {}'.format(len(pose_landmarks.landmark))
  pose_landmarks = [[lmk.x, lmk.y, lmk.z] for lmk in pose_landmarks.landmark]
  output_frame = cv2.cvtColor(output_frame, cv2.COLOR_RGB2BGR)
  frame_height, frame_width = input_frame.shape[:2]
  pose_landmarks *= np.array([frame_width, frame_height, frame_width])
f, axarr = plt.subplots(1,2, figsize = (20, 10))
axarr[0].imshow(input_frame)
axarr[1].imshow(output_frame)


In [ ]:
cap = cv2.VideoCapture(0)
# while cap.isOpened():
#     ret, frame = cap.read()
#     cv2.imshow('Mediapipe Feed', frame)
    
#     if cv2.waitKey(10) & 0xFF == ord('q'):
#         break
        
# cap.release()
# cv2.destroyAllWindows()

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [ ]:

item = pose_landmarks
arr = np.array([cal_ang_test(item, 13, 11, 12),
          cal_ang_test(item, 24, 12, 14),
          cal_ang_test(item, 15, 13, 11),
          cal_ang_test(item, 12, 14, 16),
          cal_ang_test(item, 11, 23, 25),
          cal_ang_test(item, 12, 24, 26),
          cal_ang_test(item, 23, 25, 27),
          cal_ang_test(item, 24, 26, 28)])
print(arr)
print(classifier.predict([arr]))

video tests

In [ ]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img
# JavaScript to properly create our live video stream using our webcam as input

def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  print(data)
  return data

In [ ]:
from mediapipe.python.solutions import drawing_utils as mp_drawing
from mediapipe.python.solutions import pose as mp_pose

In [ ]:
# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0

while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    frame = js_to_image(js_reply["img"])
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    with mp_pose.Pose() as pose_tracker:
        result = pose_tracker.process(image=frame)
        # mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)
        pose_landmarks = result.pose_landmarks
        print(pose_landmarks)
    output_frame = frame.copy()
    if pose_landmarks is not None:
        mp_drawing.draw_landmarks(
            image=output_frame,
            landmark_list=pose_landmarks,
            connections=mp_pose.POSE_CONNECTIONS)
        assert len(pose_landmarks.landmark) == 33, 'Unexpected number of predicted pose landmarks: {}'.format(
            len(pose_landmarks.landmark))
        pose_landmarks = [[lmk.x, lmk.y, lmk.z]
                          for lmk in pose_landmarks.landmark]
        output_frame = cv2.cvtColor(output_frame, cv2.COLOR_RGB2BGR)
        frame_height, frame_width = frame.shape[:2]
        pose_landmarks *= np.array([frame_width, frame_height, frame_width])
    # cv2.imshow('frame', output_frame)
    bbox_bytes = bbox_to_bytes(pose_landmarks)
    bbox = bbox_bytes

    # cv2_imshow(img)
    # create transparent overlay for bounding box
    # bbox_array = np.zeros([480,640,4], dtype=np.uint8)

    # grayscale image for face detection
    # gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # get face region coordinates
    # faces = face_cascade.detectMultiScale(gray)
    # get face bounding box for overlay
    # for (x,y,w,h) in faces:
    #   bbox_array = cv2.rectangle(bbox_array,(x,y),(x+w,y+h),(255,0,0),2)

    # bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # # convert overlay of bbox into bytes
    # bbox_bytes = bbox_to_bytes(bbox_array)
    # # update bbox so next frame gets new overlay
    # bbox = bbox_bytes


In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

suryanamaskar dataset


1. pranamaasna
2.   
3. Uttanasana
4. 
5. 
6. 
7. cobra pose / Urdhva Mukha Svsnssana
8. Uttanasana
9. 
10. 
11. 
12. 


dataset graphs for yoga

In [ ]:
import matplotlib.pyplot as plt

from sklearn.datasets import make_classification
from sklearn.datasets import make_blobs
from sklearn.datasets import make_gaussian_quantiles

plt.figure(figsize=(8, 8))
plt.subplots_adjust(bottom=0.05, top=0.9, left=0.05, right=0.95)

plt.subplot(321)
plt.title("One informative feature, one cluster per class", fontsize="small")
X1, Y1 = make_classification(
    n_features=2, n_redundant=0, n_informative=1, n_clusters_per_class=1
)
plt.scatter(X1[:, 0], X1[:, 1], marker="o", c=Y1, s=25, edgecolor="k")

plt.subplot(322)
plt.title("Two informative features, one cluster per class", fontsize="small")
X1, Y1 = make_classification(
    n_features=2, n_redundant=0, n_informative=2, n_clusters_per_class=1
)
plt.scatter(X1[:, 0], X1[:, 1], marker="o", c=Y1, s=25, edgecolor="k")

plt.subplot(323)
plt.title("Two informative features, two clusters per class", fontsize="small")
X2, Y2 = make_classification(n_features=2, n_redundant=0, n_informative=2)
plt.scatter(X2[:, 0], X2[:, 1], marker="o", c=Y2, s=25, edgecolor="k")

plt.subplot(324)
plt.title("Multi-class, two informative features, one cluster", fontsize="small")
X1, Y1 = make_classification(
    n_features=2, n_redundant=0, n_informative=2, n_clusters_per_class=1, n_classes=3
)
plt.scatter(X1[:, 0], X1[:, 1], marker="o", c=Y1, s=25, edgecolor="k")

plt.subplot(325)
plt.title("Three blobs", fontsize="small")
X1, Y1 = make_blobs(n_features=2, centers=3)
plt.scatter(X1[:, 0], X1[:, 1], marker="o", c=Y1, s=25, edgecolor="k")

plt.subplot(326)
plt.title("Gaussian divided into three quantiles", fontsize="small")
X1, Y1 = make_gaussian_quantiles(n_features=2, n_classes=3)
plt.scatter(X1[:, 0], X1[:, 1], marker="o", c=Y1, s=25, edgecolor="k")

plt.show()